# Лабораторна робота 1 з "Асиметричних криптосистем та протоколів"
## Тема: Побудова тестів для перевірки якості випадкових та псевдовипадкових послідовностей.

**Виконали**\
Бондар Петро, ФІ-03\
Кістаєв Матвій, ФІ-03

## Підготовка до виконання лабораторнох роботи

In [22]:
import numpy as np
import random
import secrets
import io
import scipy

N = 1000000
Alpha = 0.05

## Функція статистичних тестів

In [23]:
# Швидко не буде.
def bits_to_bytes(seq):
    n = len(seq) // 8
    res = np.zeros(n ,dtype=np.uint8)

    bit_string = "".join(map(str, list(seq)))
    res = np.array([int("0b" + bit_string[i*8:(i*8)+8], 2) for i in range(n)])

    return res

In [24]:
def test(seq: np.array, b = 8):
    n = len(seq)

    # Рівномірність
    counts = np.zeros(2**b)
    n_j = n / (2**b)

    for a in seq:
        counts[a] += 1

    stat_U = np.sum((counts - n_j)**2 / n_j)
    quant_U = scipy.stats.chi2.ppf(1 - Alpha, 2**b - 1)
    
    print("Тест на рівноймовірність символів:  " + str(stat_U <= quant_U) + "   (" + str(stat_U) + ")")


    # Незалежність символів
    pair_counts = np.zeros((2**b, 2**b), dtype=float)

    for i in range(0, n-1, 1):
        pair_counts[seq[i]][seq[i+1]] = pair_counts[seq[i]][seq[i+1]] + 1

    S = 0
    for i in range(0, 2**b):
        for j in range(0, 2**b):
            d = (np.sum(pair_counts[i, :]) * np.sum(pair_counts[:, j]))
            if d != 0:
                S += (pair_counts[i, j] ** 2) / d

    stat_I = n * (S - 1)
    quant_I = scipy.stats.chi2.ppf(1 - Alpha, (2**b - 1)**2)

    print("Тест на незалежність символів:  " + str(stat_I <= quant_I) + "   (" + str(stat_I) + ")")

    # Однорідність послідовності
    r = 200
    interval_counts = np.zeros((r, 2**b), dtype=float)

    for i in range(0, r):
        for j in range(0, n // r):
            c = seq[r*i + j]
            interval_counts[i, c] += 1

    S = 0
    for i in range(0, r):
        for j in range(0, 2**b):
            d = (np.sum(interval_counts[i, :]) * np.sum(interval_counts[:, j]))
            if d != 0:
                S += (interval_counts[i, j] ** 2) / d
    
    stat_H = n * (S - 1)
    quant_H = scipy.stats.chi2.ppf(1 - Alpha, (2**b - 1) * (r - 1))

    print("Тест на однорідність послідовності:  " + str(stat_H <= quant_H) + "   (" + str(stat_H) + ")")


## Генератори і результати їх тестування

### Вбудовані генератори

#### Криптографічно нестійкий генератор

In [46]:
print("\nВбудований генератор, нестійкий криптографічно (bits):")
unsafe_bits_seq = np.array([random.randint(0, 1) for _ in range(N)], dtype=np.uint8)
test(bits_to_bytes((unsafe_bits_seq)))

print("\nВбудований генератор, нестійкий криптографічно (bytes):")
unsafe_bytes_seq = np.array(list(random.randbytes(N)), dtype=np.uint8)
test(unsafe_bytes_seq)


Вбудований генератор, нестійкий криптографічно (bits):
Тест на рівноймовірність символів:  True   (277.421568)
Тест на незалежність символів:  True   (65192.97317313633)
Тест на однорідність послідовності:  False   (51776.0278656603)

Вбудований генератор, нестійкий криптографічно (bytes):
Тест на рівноймовірність символів:  True   (216.55859199999998)
Тест на незалежність символів:  True   (65546.31882235884)
Тест на однорідність послідовності:  True   (42979.861873307265)


#### Криптографічно стійкий генератор

In [26]:
print("\nВбудований генератор, стійкий криптографічно (bits):")
safe_bits_seq = np.array([secrets.randbelow(2) for _ in range(N)], dtype=np.uint8)
test(safe_bits_seq, 1)

print("\nВбудований генератор, стійкий криптографічно (bytes):")
safe_bytes_seq = np.array(list(secrets.token_bytes(N)), dtype=np.uint8)
test(safe_bytes_seq)


Вбудований генератор, стійкий криптографічно (bits):
Тест на рівноймовірність символів:  True   (1.1881)
Тест на незалежність символів:  True   (1.4395548948442638)
Тест на однорідність послідовності:  True   (99.96831475533874)

Вбудований генератор, стійкий криптографічно (bytes):
Тест на рівноймовірність символів:  True   (282.356224)
Тест на незалежність символів:  True   (65506.55085910772)
Тест на однорідність послідовності:  True   (46104.31010290106)


### Генератор Лемера

In [27]:
class Linear_Low:
    a = 2**16 + 1
    c = 119
    x0 = 5  # тюряга #

    def generate_bytes(self, n: int):
        seq = np.zeros(n, dtype=np.uint32)
        seq[0] = self.x0

        for i in range(0, n - 1):
            seq[i+1] = (self.a*seq[i] + self.c) 

        seq = np.array(seq % (2**8), dtype=np.uint8)

        return seq

class Linear_High:
    a = 2**16 + 1
    c = 119
    x0 = 1  # тюряга #

    def generate_bytes(self, n: int):
        seq = np.zeros(n, dtype=np.uint32)
        seq[0] = self.x0

        for i in range(0, n - 1):
            seq[i+1] = (self.a*seq[i] + self.c)

        seq = np.array(seq >> 24, dtype=np.uint8)

        return seq


In [28]:
de1 = Linear_Low()
de2 = Linear_High()

t1 = de1.generate_bytes(N)
t2 = de2.generate_bytes(N)

print("\nГенератор Лемера (Low):")
test(t1)
print()
print("\nГенератор Лемера (High):")
test(t2)


Генератор Лемера (Low):
Тест на рівноймовірність символів:  True   (0.012288)
Тест на незалежність символів:  False   (255000000.0)
Тест на однорідність послідовності:  True   (652.771437011701)


Генератор Лемера (High):
Тест на рівноймовірність символів:  True   (23.938560000000003)
Тест на незалежність символів:  True   (63474.788106759435)
Тест на однорідність послідовності:  True   (45609.157861016494)


### Генератор L20

In [29]:
class L20:
    def __init__(self, x_init: np.array):
        self.x_init = x_init

    def generate_bits(self, n: int):
        seq = np.concatenate([np.array(self.x_init, dtype=np.uint8), np.zeros(n - 20, dtype=np.uint8)])

        for i in range(20, n):
            seq[i] = seq[i - 3] ^ seq[i - 5] ^ seq[i - 9] ^ seq[i - 20]

        return seq
            

In [30]:
smp = np.array([random.randint(0, 1) for _ in range(20)])

de_L20 = L20(smp)

print("\nГенератор L20 (1M bits):")
print(f"Початкове заповнення: {smp}")
test(bits_to_bytes(de_L20.generate_bits(N)))


print("\nГенератор L20 (16M bits):")
test(bits_to_bytes(de_L20.generate_bits(16*N)))


Генератор L20 (1M bits):
Початкове заповнення: [0 1 0 1 0 1 0 0 0 1 1 1 1 0 1 0 0 0 0 1]
Тест на рівноймовірність символів:  True   (232.472064)
Тест на незалежність символів:  True   (57431.758789527106)
Тест на однорідність послідовності:  True   (50119.11594191967)

Генератор L20 (16M bits):
Тест на рівноймовірність символів:  True   (11.421696)
Тест на незалежність символів:  True   (2875.761178746661)
Тест на однорідність послідовності:  True   (39428.55462109662)


### Генератор L89

In [31]:
class L89:
    def __init__(self, x_init: np.array):
        self.x_init = x_init

    def generate_bits(self, n: int):
        seq = np.concatenate([np.array(self.x_init, dtype=np.uint8), np.zeros(n - 89, dtype=np.uint8)])

        for i in range(89, n):
            seq[i] = seq[i - 38] ^ seq[i - 89]

        return seq
            

In [32]:
smp = np.array([random.randint(0, 1) for _ in range(89)])
de_L89 = L89(smp)

print("\nГенератор L89 (1M bits):")
print(f"Початкове заповнення: {smp}")
test(bits_to_bytes(de_L89.generate_bits(N)))


Генератор L89 (1M bits):
Початкове заповнення: [0 1 0 0 1 0 0 1 0 0 0 0 1 1 0 1 0 1 0 1 0 0 1 1 1 0 0 0 1 0 0 0 1 1 0 0 1
 1 0 1 0 1 0 0 0 1 1 1 0 0 0 0 0 0 1 1 1 1 1 0 1 1 1 0 0 1 1 1 0 0 1 0 0 1
 0 1 0 1 0 1 1 1 1 0 0 0 1 1 1]
Тест на рівноймовірність символів:  True   (271.277568)
Тест на незалежність символів:  True   (64926.63650964034)
Тест на однорідність послідовності:  True   (49770.095734737064)


### Генератор Джиффі

In [33]:
class Geffe:
    def __init__(self, x_init: np.array, y_init: np.array, s_init: np.array):
        self.x = x_init # 11 bits, x11 = x0 + x2
        self.y = y_init # 9 bits,  y9 = y0 + y1 + y3 + y4
        self.s = s_init # 10 bits, s10 = s0 + s3

    def generate_bits(self, n: int):
        seq = np.zeros(n, dtype=np.uint8)

        for i in range(0, n):
            seq[i] = (self.s[0] * self.x[0]) ^ ((1 ^ self.s[0]) * self.y[0])
            # Linear Shift
            self.x[0] = self.x[0] ^ self.x[2]
            self.x = np.roll(self.x, -1)
            self.y[0] = self.y[0] ^ self.y[1] ^ self.y[3] ^ self.y[4]
            self.y = np.roll(self.y, -1)
            self.s[0] = self.s[0] ^ self.s[3]
            self.s = np.roll(self.s, -1)

        return seq
            

In [34]:
x = np.array([random.randint(0, 1) for _ in range(11)])
y = np.array([random.randint(0, 1) for _ in range(9)])
s = np.array([random.randint(0, 1) for _ in range(10)])


Ge_generator = Geffe(x, y, s)

print("\nГенератор Джиффі:")
print(f"x = {x}")
print(f"y = {y}")
print(f"s = {s}")

test(bits_to_bytes(Ge_generator.generate_bits(N)))


Генератор Джиффі:
x = [0 0 0 0 1 1 1 0 0 1 1]
y = [0 0 0 1 0 0 1 1 1]
s = [1 0 1 1 1 1 1 0 0 0]
Тест на рівноймовірність символів:  True   (226.93427199999996)
Тест на незалежність символів:  False   (79396.11800166707)
Тест на однорідність послідовності:  True   (49294.79021238856)


### Генератор "Бібліотекар"

In [35]:
class Librarian:
    def __init__(self, filename):
        file = io.open(filename, mode='r', encoding='utf-8')
        self.text = file.read()

    
    def generate_bytes(self, n: int):
        if len(self.text) < n:
            raise RuntimeError("Nema sliv, odni emotions")

        seq = np.zeros(n, dtype=np.uint8)

        for i in range(0, n):
            seq[i] = (ord(self.text[i]) % 2**8)

        return seq
            

In [36]:
de_Lb = Librarian("fanfiction.txt")

print("\nГенератор \"Бібліотекар\":")
test(de_Lb.generate_bytes(N))


Генератор "Бібліотекар":
Тест на рівноймовірність символів:  False   (15620446.022144001)
Тест на незалежність символів:  False   (4077309.429118166)
Тест на однорідність послідовності:  True   (33138.23958738649)


### Генератор Вольфрама

In [37]:
# В ПІТОНІ НЕМА ВБУДОВАНОГО ЦИКЛІЧНОГО ЗСУВУ
def rcs(n: np.uint32, rotations) -> np.uint32: 
    return (n >> rotations | n << (32-rotations)) % 2**32

def lcs(n: np.uint32, rotations) -> np.uint32:
    return (n << rotations | n >> (32-rotations)) % 2**32

class Wolfram:
    def __init__(self, r0: np.uint32):
        self.r0 = r0

    def generate_bits(self, n: int):
        r_i = self.r0
        seq = np.zeros(n, dtype=np.uint8)

        for i in range(0, n):
            seq[i] = r_i % 2
            r_i = lcs(r_i, 1) ^ (r_i | rcs(r_i, 1))

        return seq
            

In [38]:
de_wolfram = Wolfram(1)

print("\nГенератор Вольфрама:")
test(bits_to_bytes(de_wolfram.generate_bits(N)))


Генератор Вольфрама:
Тест на рівноймовірність символів:  False   (309.34988799999996)
Тест на незалежність символів:  False   (81381.05684421698)
Тест на однорідність послідовності:  True   (50802.358118155535)


### Генератор BM

In [39]:
class BM:
    def __init__(self, p, a):
        self.p = p
        self.a = a

    def generate_bits(self, n: int):
        seq = np.zeros(n, dtype=object)
        seq[0] = random.randint(0, self.p - 1) 

        for i in range(1, n):
            seq[i] = pow(self.a, seq[i - 1], self.p)

        seq = np.array(seq < (self.p - 1) / 2, dtype=np.uint8) 

        return seq
    
    def generate_bytes(self, n: int):
        seq = np.zeros(n, dtype=object)
        seq[0] = random.randint(0, self.p - 1) 

        for i in range(1, n):
            seq[i] = pow(self.a, seq[i - 1], self.p)

        seq = np.array((seq * 256) // (self.p - 1), dtype=np.uint8) 

        return seq
            

In [40]:
p = int("CEA42B987C44FA642D80AD9F51F10457690DEF10C83D0BC1BCEE12FC3B6093E3", 16)
a = int("5B88C41246790891C095E2878880342E88C79974303BD0400B090FE38A688356", 16)

de_BM = BM(p, a)

print("\nГенератор BM (bits):")
test(bits_to_bytes(de_BM.generate_bits(N)))

print("\nГенератор BM (bytes):")
test(de_BM.generate_bytes(N))


Генератор BM (bits):
Тест на рівноймовірність символів:  True   (272.809472)
Тест на незалежність символів:  True   (65237.96210366378)
Тест на однорідність послідовності:  True   (50498.63139488933)

Генератор BM (bytes):
Тест на рівноймовірність символів:  True   (253.33555199999998)
Тест на незалежність символів:  True   (64818.830484185906)
Тест на однорідність послідовності:  True   (44917.441824667614)


### Генератор BBS

In [41]:
class BBS:
    def __init__(self, p, q):
        self.n = p*q

    def generate_bits(self, n: int):
        seq = np.zeros(n, dtype=object)
        seq[0] = random.randint(2, self.n - 1) 

        for i in range(1, n):
            seq[i] = pow(seq[i - 1], 2, self.n)

        seq = np.array(seq % 2, dtype=np.uint8) 

        return seq
    
    def generate_bytes(self, n: int):
        seq = np.zeros(n, dtype=object)
        seq[0] = random.randint(2, self.n - 1) 

        for i in range(1, n):
            seq[i] = pow(seq[i - 1], 2, self.n)

        seq = np.array(seq % (2**8), dtype=np.uint8) 

        return seq
            

In [42]:
p = int("D5BBB96D30086EC484EBA3D7F9CAEB07", 16)
q = int("425D2B9BFDB25B9CF6C416CC6E37B59C1F", 16)

de_BBS = BBS(p, q)

print("\nГенератор BSS (bits):")
test(bits_to_bytes(de_BBS.generate_bits(N)))

print("\nГенератор BSS (bytes):")
test(de_BBS.generate_bytes(N))


Генератор BSS (bits):
Тест на рівноймовірність символів:  True   (243.920384)
Тест на незалежність символів:  True   (64496.78124527719)
Тест на однорідність послідовності:  True   (49830.8976887267)

Генератор BSS (bytes):
Тест на рівноймовірність символів:  True   (255.72454399999998)
Тест на незалежність символів:  True   (64919.47539325205)
Тест на однорідність послідовності:  True   (42669.42277055463)


## Результати статистичного оцінювання генераторів

За взятого $\alpha = 0.05$:

$$\begin{array}{|l|c|c|c|}
\hline \text{Генератор} & \text{Рівноймовірність символів} & \text{Незалежність символів} & \text{Однорідність послідовності}\\
\hline \text{random bits} & \checkmark & \checkmark & X\\
\hline \text{random bytes} & \checkmark & \checkmark & \checkmark\\
\hline \text{secret bits} & \checkmark & \checkmark & \checkmark\\
\hline \text{secret bytes} & \checkmark & \checkmark & \checkmark\\
\hline \text{LehmerLow} & \checkmark & X & \checkmark\\
\hline \text{LehmerHigh} & \checkmark & \checkmark & \checkmark\\
\hline \text{L20} & \checkmark & \checkmark & \checkmark\\
\hline \text{L89} & \checkmark & \checkmark & \checkmark\\
\hline \text{Джиффі} & \checkmark & \checkmark & \checkmark\\
\hline \text{"Бібліотекар"} & X & X & \checkmark\\
\hline \text{Вольфрам} & X & X & \checkmark\\
\hline \text{BM} & \checkmark & \checkmark & \checkmark\\
\hline \text{BM bytes} & \checkmark & \checkmark & \checkmark\\
\hline \text{BBS} & \checkmark & \checkmark & \checkmark\\
\hline \text{BBS bytes} & \checkmark & \checkmark & \checkmark\\
\hline
\end{array}$$


## Висновок
- Не бажано використовувати генератори Вольфрама і "Бібліотекаря".
- Важливо дивитися не тільки на результати тестів, а й на значення статистики для згенерованої послідовності, бо воно насправді може бути дуже близьким до межі.
- Обережно користуйтесь інтами в пітоні, вони можуть вибухнути.